In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer

/home/aastroza/miniconda3/envs/ids-gpt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"

In [4]:
class Model:
    def init_model(self):
        self.model = SentenceTransformer(
            EMBEDDING_MODEL_NAME,
        )

    def __enter__(self):
        self.init_model()

    def __exit__(self, *args):
        pass

    def uncached_embed(self, texts: list[str]) -> list[str]:
        if not hasattr(self, "model"):
            self.init_model()
        embeddings = self.model.encode(texts, show_progress_bar=True)
        return embeddings

In [5]:
model = Model()

In [97]:
df = pd.read_csv('data/raw/constitucion_vigente_corregida.csv')

In [101]:
df['texto_con_titulo'] = df['titulo_capitulo'] + '\n' + df['texto']

In [104]:
pred1 = model.uncached_embed(list(df['texto']))
pred2 = model.uncached_embed(list(df['texto_con_titulo']))

Batches: 100%|██████████| 11/11 [00:00<00:00, 28.51it/s]


In [105]:
df['emb_texto'] = list(pred1)
df['emb_texto_con_titulo'] = list(pred2)

In [106]:
df.to_csv('data/processed/constitucion_vigente_embeddings.csv')

In [25]:
from pydantic import BaseModel
from typing import Dict, List

class Similarity(BaseModel):
    source: str
    target: List[str]


class SimilarityResponse(BaseModel):
    targetIndex: int
    sentenceIndex: int


In [63]:
df_gh = pd.read_csv('https://raw.githubusercontent.com/idsudd/discolab/main/data/processed/constitucion_vigente_embeddings.csv?token=GHSAT0AAAAAAB7EZLAPM2B5TFSNJWRYFEYWZDGZERQ')

In [58]:
#df_gh['emb'].apply(lambda x: np.array(eval(x)), 0)

result = df_gh['emb'].apply(lambda x: 
                           np.fromstring(
                               x.replace('\n','')
                                .replace('[','')
                                .replace(']','')
                                .replace('  ',' '), sep=' '))

In [59]:
result

0      [-0.0389939658, 0.0242968816, -0.0595533401, -...
1      [0.0186691079, 0.0752222538, -0.0529866517, -0...
2      [0.103690401, -0.0504060499, -0.0535255224, -0...
3      [-0.0057254592, 0.00908108521, -0.121637784, 0...
4      [0.0800063759, -0.00759706181, -0.0282481089, ...
                             ...                        
320    [0.0613458715, 0.0283422749, -0.00767591456, -...
321    [0.0183023494, 0.0172349997, -0.00242855749, -...
322    [0.145207107, 0.033818271, -0.0132987155, -0.0...
323    [0.0794083476, 0.0504507646, -0.099090904, -0....
324    [0.0349203274, 0.0565979704, -0.0391424932, -0...
Name: emb, Length: 325, dtype: object

In [93]:
import numpy as np
df_gh = pd.read_csv('https://raw.githubusercontent.com/idsudd/discolab/main/data/processed/constitucion_vigente_embeddings.csv')

preds = df_gh['emb'].apply(lambda x: 
                           np.fromstring(
                               x.replace('\n','')
                                .replace('[','')
                                .replace(']','')
                                .replace('  ',' '), sep=' '))
query = 'El banco central'
target = model.uncached_embed([query])[0]

similarities = np.dot(np.array(list(preds)), np.array(target).T)
ind = np.argpartition(similarities, -5)[-5:]

df_array = []
for i, row in df.loc[ind].iterrows():
    df_array.append(df_gh[(df_gh['capitulo'] == row['capitulo'])&(df_gh['articulo'] == row['articulo'])].reset_index(drop=True))

Batches: 100%|██████████| 1/1 [00:00<00:00, 116.24it/s]


In [91]:
df_array

[    Unnamed: 0    capitulo    titulo_capitulo     articulo  numero   
 0          149  Capítulo V  CONGRESO NACIONAL  Artículo 63       0  \
 1          150  Capítulo V  CONGRESO NACIONAL  Artículo 63       1   
 2          151  Capítulo V  CONGRESO NACIONAL  Artículo 63       2   
 3          152  Capítulo V  CONGRESO NACIONAL  Artículo 63       3   
 4          153  Capítulo V  CONGRESO NACIONAL  Artículo 63       4   
 5          154  Capítulo V  CONGRESO NACIONAL  Artículo 63       5   
 6          155  Capítulo V  CONGRESO NACIONAL  Artículo 63       6   
 7          156  Capítulo V  CONGRESO NACIONAL  Artículo 63       7   
 8          157  Capítulo V  CONGRESO NACIONAL  Artículo 63       8   
 9          158  Capítulo V  CONGRESO NACIONAL  Artículo 63       9   
 10         159  Capítulo V  CONGRESO NACIONAL  Artículo 63      10   
 11         160  Capítulo V  CONGRESO NACIONAL  Artículo 63      11   
 12         161  Capítulo V  CONGRESO NACIONAL  Artículo 63      12   
 13   

In [96]:
df_result = pd.concat(df_array, ignore_index=True)
df_result.drop_duplicates(ignore_index=True).sort_values(by = ['Unnamed: 0'])

,Unnamed: 0,capitulo,titulo_capitulo,articulo,numero,texto,emb
22,133,Capítulo V,CONGRESO NACIONAL,Artículo 57,0,No pueden ser candidatos a diputados ni a sena...,[ 8.73802416e-03 4.50450741e-02 -4.02881540e-...
23,134,Capítulo V,CONGRESO NACIONAL,Artículo 57,1,1) Los Ministros de Estado;,[ 8.45800713e-02 7.34119639e-02 -1.40742457e-...
24,135,Capítulo V,CONGRESO NACIONAL,Artículo 57,2,"2) Los gobernadores regionales, los delegados ...",[ 1.20025471e-01 -2.60943864e-02 7.84167126e-...
25,136,Capítulo V,CONGRESO NACIONAL,Artículo 57,3,3) Los miembros del Consejo del Banco Central;,[ 3.64259183e-02 -4.56845574e-02 -3.08456644e-...
26,137,Capítulo V,CONGRESO NACIONAL,Artículo 57,4,4) Los magistrados de los tribunales superiore...,[ 2.64500901e-02 6.96221665e-02 -5.72755607e-...
27,138,Capítulo V,CONGRESO NACIONAL,Artículo 57,5,"5) Los miembros del Tribunal Constitucional, d...",[ 1.7351098e-02 2.0509174e-02 -1.2341870e-02 ...
28,139,Capítulo V,CONGRESO NACIONAL,Artículo 57,6,6) El Contralor General de la República;,[-4.91214842e-02 6.47928342e-02 2.89518051e-...
29,140,Capítulo V,CONGRESO NACIONAL,Artículo 57,7,7) Las personas que desempeñan un cargo direct...,[-5.50116738e-03 4.94735204e-02 -5.95650114e-...
30,141,Capítulo V,CONGRESO NACIONAL,Artículo 57,8,8) Las personas naturales y los gerentes o adm...,[ 2.83397436e-02 1.08643323e-01 2.53515206e-...
31,142,Capítulo V,CONGRESO NACIONAL,Artículo 57,9,"9) El Fiscal Nacional, los fiscales regionales...",[ 2.10502204e-02 -3.58795598e-02 1.51831354e-...
